# 5.6	Step VI: Link budget feasibility assessment: How much data can we download?

To assess the link budget, both in terms of payload data generated and in terms of download capability (in time), an orbital simulation of the satellites, ground segment and area of interest is required. In this section some of those scenarios are presented, to indicate the though process). Each simulation was ran for 30 days starting January 1st, 2025, it took 90-100 minutes to complete.

The scenarios where ran with three ground stations and the link assessed for one, any two or all three stations to accommodate the data generated by the satellites. The stations chosen are:

- Cuiaba (Brazil) prime station in the stakeholder’s country
    - Latitude: -23.2°
    - Longitude: -45.87°

- Punta Arenas (Chile)
    - Latitude: -52.93°
    - Longitude: -70.85°
    
- Svalbard (Norway)
    - Latitude: 78.23°
    - Longitude: 15.38°

Both Punta Arenas and Svalbard have KSat lite capabilities, as discussed above the ground segment as a service is a relevant strategy for cost-effective missions. All stations are presented in Figure 5 9.


Figure 5-9 Orbital scenario ground station access areas.
!['Ground Stations'](Figs/2DMap%20with%20GS.png)
Source: the author.

The data shown on these graphs are relative to one satellite in the cluster, called Walker_1 in the files. In the previous image the three main ground stations used in the simulation are shown. To remove the short connections between the satellites and the ground stations, a filter is implemented towards the data obtained such as links are only valid if they are five minutes long or more. The later graphs represent the total amount of data produced daily by the cluster when passing the area, the total amount of data available to download by the ground stations and finally the total amount of link time per day on each ground station of all satellites.

## 5.6.1 Simulation setup

The requirement it to have a certain point within the area of interest (AoI) covered (i.e. observed by the payload on board any of the satellites) twice a day (see requirements Table 5-1).

The model implemented in Free Flyer works in the following sequence:

- Define and Area of Interest (for our case a subset of Amazonia defined in Figure 5-1) with a surface of $1615668\ km^2$. Estimated a priori through a polygon in Google Earth see Figure 5-10.

Figure 5-10 Screenshot of preliminary surface in area of interest.
!["Preliminary surface"](Figs/Area%20AOI.png)
Source: the author.

- Define the number of points within the AoI (discretizes the area of interest by the number of "points" resolution set up). For our case we use 40984, which divides the AoI into $39.4\ km^2$, see Figure 5-11, yields a surface per point within the area of interest of: 

$$	
S_{AoI}=\frac{1615668}{40984}[km^2]=39.4km2
$$


Figure 5-11 Overview of the discretization of the AoI (in 39.4 $km^2$ points)
!["Overview"](Figs/Captura%20de%20pantalla%202024-05-03%20151205.png)
Source: the author.

From literature, swath width of 20 km exists, and thus this discretization would be a $20\left[km\right]\times\ 2\ \left[km\right]=40[km^2]$ scene which is feasible by analogy. 
Based on the requirements from FireSat-II the number of passes on the designated area shall be at least twice per day. Thus, an iterative process is executed where the main parameters to trade are the number of satellites, the spatial resolution in the form of the Field of View (FOV) (or in Freeflyer the so called "half angle", half the FOV), and the type of orbit (between ISS and SSO and their altitudes).



### 5.6.1.1 Field of View 

The Field of View (FOV) corresponds to the solid angle through which a detector element (a pixel sensor) is sensitive to electromagnetic radiation at any one time. In that case is called instantaneous field of view or IFOV. A measure of the spatial resolution of a remote sensing imaging system, it is often expressed as dimensions of visible ground area, for some known sensor altitude. For this analysis scenario it is established that the view of the satellite will not be based on a cone but rather a rectangle, using the CAM sensor used by the KITSUNE 6U spacecraft (BIN AZAMI et al., 2021) . After that, the swath width can be estimated.
We begin by the spatial resolutions provided previously $R_{GSD}$. With this information you can estimate the swath width by multiplying it with sizes of $6464\times\ 4852\ [px]$ based on Kitsune (AZAMI et al., 2022b). Then, based on the sizing relationships by (WERTZ; EVERETT; PUSCHELL, 2011a), you can calculate the half angles of the FOV of the sensor pointing nadir with Eq. 5-6, Eq. 5-7 and Eq. 5-8 where the variables are represented in Figure 5-12 with $\alpha_a$,$\ \alpha_b$ are the angles corresponding the width and height measured from the center lines respectively, $x_a$,$\ x_b$ are the half width and half height of the sensor respectively.

 
$$ S_W\ =\ 2h\arctan(\alpha_a)\ [m]$$
Eq. 5-6

 
$$\alpha_a=\tan^{-1}{\left(\frac{x_a}{f}\right)}\ \ [°]$$ 
Eq. 5-7

$$\alpha_b=\tan^{-1}{\left(\frac{x_b}{f}\right)}\ \ [°]$$ 
Eq. 5-8

Figure 5-12 Nomenclature of the sensor’s field of view and parameters to be calculated.
!["Nomenclature"](Figs/Field%20of%20View%20Nomenclature%20Diagram.png)
Source: the author.

The previous equations are implemented in Python and used to provide a wide array of angles and focal lengths for the different orbital scenarios to be inputted into the Free Flyer orbital simulation. The results show in Table 5-4 to Table 5-8 for different altitudes. Where the results also provide input and requirements for the payload to be designed (as the focal length of the telescope is preliminary assessed). In the previous work by LOPEZ-TELGIE; DOS SANTOS,  (2020a), which is based on the work of Valenzuela and Reyes (2019c), and “educated guesses” on focal length for different satellites (based upon discussion with satellite manufactures) the values for spatial resolution using different metrics where estimated (see section 2.7.1)

In [11]:
import numpy as np
import pandas as pd
# Spatial resolution is required to be 50 [m] Req 34.1.2

R_GSD= np.array([5, 10, 20, 25, 50]) # [m] 
# Ground Sample Distance 25 [m] is used for 1st iteration considering some image quality considerations in CubeSat telescopes.

h = np.array([2.75e5, 3.5e5, 4.6e5, 4.5e5, 6e5]) #[m] the mission altitudes 

S_w= 6464 * R_GSD # [m] Estimated from equivalent CubeSat for fire monitoring mission. The 6464 value is the amount of pixels in the width of the sensor used by KITSUNE's CAM [Asamis et al, 2022].


#initializing arrays
alpha_a = np.zeros((len(h), len(R_GSD)))
f = np.zeros((len(h), len(R_GSD)))
alpha_b = np.zeros((len(h), len(R_GSD)))


px_size = 3.4736e-6 # [m] estimated from Eq. 1 and data provided by [Asif et al. 2022].
#it is calculated with the focal length of 300 mm of KITSUNE's CAM, and either the 4.4 [m] in R_GSD at 380 [km] or 4.8 [m] at 420 [km]. Then, Eq. 1 is used to get px_size.

x_a = 6464 *px_size / 2 # [m] size of the half of the sensor's width [Azami et al., 2022]
x_b = 4852 *px_size / (2*4) # [m] size of the half of the sensor's height [Azami et al., 2022].
# Assuming panchromatic sensor and NIR present, the sensor will be split evenly on it height 4 times.

diferencias = {}

for i, r in enumerate(h):
    diff = np.empty((0,4))
    for resolution in range(len(R_GSD)): # the equations shown above to calculate the afore mentioned.
        
        alpha_a = np.degrees(np.arctan(S_w[resolution] / (2*r))) # width half angle
        f = x_a / np.tan(np.radians(alpha_a)) #focal length
        alpha_b =  np.degrees(np.arctan(x_b/f)) #height half angle

        diff = np.vstack((diff, [R_GSD[resolution], np.round(alpha_a, 2), np.round(alpha_b, 2), np.round(f*1000,2)]))
    
    diferencias[f'{i+1}, h= {r/1000} [km]'] = pd.DataFrame(diff, columns=['Ground Sample Distance R_GSD [m]',
                                                              'Half Width Angle alpha_a [°]',
                                                              'Half Height Angle alpha_b [°]',
                                                              'Instrumental Focal Length f [mm]']) #build the tables

counter = 0
# Display
for key, df in diferencias.items():
    print("Table 5 - " + str(counter + 4) + " Sensor sizing parameter for " + str(h[counter]/1000) + "km altitude.:")
    counter += 1
    display(df)
    print()

Table 5 - 4 Sensor sizing parameter for 275.0km altitude.:


,Ground Sample Distance R_GSD [m],Half Width Angle alpha_a [°],Half Height Angle alpha_b [°],Instrumental Focal Length f [mm]
0,5.0,3.36,0.63,191.05
1,10.0,6.70,1.26,95.52
2,20.0,13.23,2.53,47.76
3,25.0,16.37,3.16,38.21
4,50.0,30.44,6.29,19.10



Table 5 - 5 Sensor sizing parameter for 350.0km altitude.:


,Ground Sample Distance R_GSD [m],Half Width Angle alpha_a [°],Half Height Angle alpha_b [°],Instrumental Focal Length f [mm]
0,5.0,2.64,0.50,243.15
1,10.0,5.28,0.99,121.58
2,20.0,10.46,1.98,60.79
3,25.0,13.00,2.48,48.63
4,50.0,24.78,4.95,24.32



Table 5 - 6 Sensor sizing parameter for 460.0km altitude.:


,Ground Sample Distance R_GSD [m],Half Width Angle alpha_a [°],Half Height Angle alpha_b [°],Instrumental Focal Length f [mm]
0,5.0,2.01,0.38,319.57
1,10.0,4.02,0.76,159.79
2,20.0,8.00,1.51,79.89
3,25.0,9.96,1.89,63.91
4,50.0,19.36,3.77,31.96



Table 5 - 7 Sensor sizing parameter for 450.0km altitude.:


,Ground Sample Distance R_GSD [m],Half Width Angle alpha_a [°],Half Height Angle alpha_b [°],Instrumental Focal Length f [mm]
0,5.0,2.06,0.39,312.62
1,10.0,4.11,0.77,156.31
2,20.0,8.17,1.54,78.16
3,25.0,10.18,1.93,62.52
4,50.0,19.75,3.86,31.26



Table 5 - 8 Sensor sizing parameter for 600.0km altitude.:


,Ground Sample Distance R_GSD [m],Half Width Angle alpha_a [°],Half Height Angle alpha_b [°],Instrumental Focal Length f [mm]
0,5.0,1.54,0.29,416.83
1,10.0,3.08,0.58,208.42
2,20.0,6.15,1.16,104.21
3,25.0,7.67,1.45,83.37
4,50.0,15.07,2.89,41.68


The results show in Table 5 4 to Table 5 8 for different altitudes. Where the results also provide input and requirements for the payload to be designed (as the focal length of the telescope is preliminary assessed). In the previous work by LOPEZ-TELGIE; DOS SANTOS, (2020a), which is based on the work of Valenzuela and Reyes (2019c), and “educated guesses” on focal length for different satellites (based upon discussion with satellite manufactures) the values for spatial resolution using different metrics where estimated (see section 2.7.1).

### 5.6.1.2	Scenario input parameters

In Table 5 9 the main scenario parameters i.e. the variables that are used to set the simulation on Free Flyer, as well as obtaining the amount of half angle regarding the swath. Two orbit types will be used with different altitudes, the Sun Synchronous Orbit (SSO) with and inclination estimated in Step IV (Section 5.4) and the orbit of the International Space Station (ISS) with an inclination of 51.64°.

Table 5-9 Orbital scenarios description and main orbital parameters.

| Scenario | Type of Orbit | # S/C | Altitude [km] | Inclination | Ground Sample Distance |
|:--------:|:-------------:|:-----:|:-------------:|:-----------:|:----------------------:|
| 1 | SSO | 10 | 450 |	97.21°	| 25 m |
| 2	| SSO | 10 | 600 |	97.87°	| 25 m |
| 3	| ISS | 10 | 275 |	51.64°	| 25 m |
| 4	| ISS | 10 | 350 |	51.64°	| 25 m |
| 5	| ISS |	10 | 460 |	51.64°	| 25 m | 
| 6	| SSO |	2  | 600 |	97.87°	| 50m |
| 7	| SSO |	5  | 600 |	97.87°	| 50m |
| 8	| SSO |	10 | 600 |	97.87°	| 50m |

Source: author.

And example of the ISS orbit is presented in Figure 5-13, and of SSO in Figure 5-14.

Figure 5-13 ISS orbit in 2D.

!["ISS Orbit"](Figs/ISS%20orbit%20example.png)

Source: the author.


Figure 5-14 SSO orbit in 2D.


!["SSO Orbit"](Figs/SSO%20orbit%20example.png)

Source: the author.

